# Triple-Extraktor für frühneuzeitliche Briefe

**Version:** 0.0.0.1

Dieses Notebook extrahiert semantische Triples aus frühneuzeitlichen Briefen (1750-1809) mit Fokus auf Konzept-Hierarchien.

In [ ]:
#@title **Setup** (einmal ausführen) { display-mode: "form" }
#@markdown Installiert benötigte Pakete und lädt den Code.

# Version
NOTEBOOK_VERSION = "0.0.0.1"
print(f"📓 Triple-Extraktor Notebook v{NOTEBOOK_VERSION}")
print(f"   Letzte Aktualisierung: 05.02.2026\n")

import sys
import os

# Pakete installieren
!pip install -q pyyaml requests plotly networkx kaleido==0.2.1

# Arbeitsverzeichnis erstellen
!mkdir -p /content/triple-colab/src
!mkdir -p /content/triple-colab/output_json
!mkdir -p /content/triple-colab/logs

# Python-Path erweitern
if '/content/triple-colab/src' not in sys.path:
    sys.path.insert(0, '/content/triple-colab/src')

# file_client.py erstellen
file_client_code = '''"""Datei-Client für XML/TXT-Dateien mit TEI-Optimierung."""
import os
import xml.etree.ElementTree as ET
from typing import Optional
import logging

logger = logging.getLogger(__name__)

class FileClient:
    """Client zum Lesen von XML/TXT-Dateien mit TEI-Optimierung."""
    
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.file_extension = os.path.splitext(file_path)[1].lower()
    
    def read_content(self) -> Optional[str]:
        """Liest den Dateiinhalt und extrahiert relevanten Text."""
        try:
            if self.file_extension == ".xml":
                return self._read_xml()
            elif self.file_extension == ".txt":
                return self._read_txt()
            else:
                logger.warning(f"Nicht unterstütztes Dateiformat: {self.file_extension}")
                return None
        except Exception as e:
            logger.error(f"Fehler beim Lesen von {self.file_path}: {e}")
            return None
    
    def _read_xml(self) -> Optional[str]:
        """Liest XML-Datei und extrahiert TEI-Body-Text."""
        try:
            tree = ET.parse(self.file_path)
            root = tree.getroot()
            namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}
            body = root.find(".//tei:body", namespaces)
            if body is not None:
                return self._extract_text_from_element(body)
            body = root.find(".//body")
            if body is not None:
                return self._extract_text_from_element(body)
            return self._extract_text_from_element(root)
        except ET.ParseError as e:
            logger.error(f"XML-Parse-Fehler in {self.file_path}: {e}")
            return None
    
    def _read_txt(self) -> Optional[str]:
        """Liest TXT-Datei."""
        with open(self.file_path, "r", encoding="utf-8") as f:
            return f.read()
    
    def _extract_text_from_element(self, element) -> str:
        """Extrahiert rekursiv Text aus XML-Element."""
        texts = []
        if element.text:
            texts.append(element.text.strip())
        for child in element:
            texts.append(self._extract_text_from_element(child))
            if child.tail:
                texts.append(child.tail.strip())
        return " ".join(filter(None, texts))
'''

with open('/content/triple-colab/src/file_client.py', 'w') as f:
    f.write(file_client_code)

# openwebui_client.py erstellen
openwebui_client_code = '''"""OpenWebUI-kompatibler Client mit Gemini/OpenAI-Support."""
import requests
import time
import logging
from typing import Optional

logger = logging.getLogger(__name__)

class OpenWebUIClient:
    """Client für OpenAI-kompatible APIs (inkl. Gemini)."""
    
    def __init__(self, api_key: str, base_url: str, model: str, temperature: float = 0.1):
        self.api_key = api_key
        self.base_url = base_url.rstrip("/")
        self.model = model
        self.temperature = temperature
        self.max_retries = 3
        self.retry_delay = 2
    
    def generate_response(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Generiert Antwort mit automatischem Retry."""
        for attempt in range(self.max_retries):
            try:
                if "generativelanguage.googleapis.com" in self.base_url:
                    return self._call_gemini_api(prompt, system_prompt)
                else:
                    return self._call_openai_api(prompt, system_prompt)
            except Exception as e:
                logger.warning(f"Versuch {attempt + 1}/{self.max_retries} fehlgeschlagen: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay * (attempt + 1))
                else:
                    logger.error(f"Alle Versuche fehlgeschlagen: {e}")
                    return None
    
    def _call_gemini_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft Gemini API auf."""
        url = f"{self.base_url}:generateContent?key={self.api_key}"
        payload = {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": self.temperature}
        }
        if system_prompt:
            payload["systemInstruction"] = {"parts": [{"text": system_prompt}]}
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["candidates"][0]["content"]["parts"][0]["text"]
    
    def _call_openai_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft OpenAI-kompatible API auf."""
        url = f"{self.base_url}/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature
        }
        response = requests.post(url, headers=headers, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]
'''

with open('/content/triple-colab/src/openwebui_client.py', 'w') as f:
    f.write(openwebui_client_code)

print("✓ Setup abgeschlossen!")
print("✓ Pakete installiert")
print("✓ Code-Module erstellt")

In [ ]:
#@title **API-Konfiguration** { display-mode: "form" }

#@markdown ### Wähle deinen API-Provider:
api_provider = "Gemini (Google)" #@param ["Gemini (Google)", "ChatAI (AcademicCloud)"]

#@markdown ### API-Schlüssel:
#@markdown **Wichtig:** Gib niemals deinen API-Schlüssel in öffentlichen Notebooks weiter! Lösche ihn vor dem Teilen.
api_key = "" #@param {type:"string"}

#@markdown ### Eigenes Modell (optional):
#@markdown Leer lassen für Standard-Modell des Providers
custom_model = "" #@param {type:"string"}

#@markdown ### Detailgrad der Extraktion:
#@markdown Wie ausführlich sollen die Triples sein? (1=nur Kernaussagen, 10=sehr detailliert)
granularity = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown ### Temperatur (Kreativität):
#@markdown Wie kreativ soll die KI antworten? (0=deterministisch, 1=sehr kreativ)
temperature = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}

#@markdown ### Erweiterte Optionen:
#@markdown Datei-Metadaten in Ergebnissen speichern (Dateiname, Datum, etc.):
save_file_metadata = True #@param {type:"boolean"}
#@markdown Debug-Modus: Zeige KI-Antworten bei Fehlern (hilft bei Problemanalyse):
show_debug_output = False #@param {type:"boolean"}

# API-Schlüssel validieren
if not api_key or len(api_key.strip()) < 10:
    raise ValueError(
        "❌ Ungültiger API-Schlüssel!\n\n"
        "Bitte gib einen gültigen API-Schlüssel ein.\n\n"
        "API-Schlüssel erhalten:\n"
        "• Gemini: https://aistudio.google.com/apikey\n"
        "• ChatAI: https://chat-ai.academiccloud.de/"
    )

# Konfiguration basierend auf Provider
if api_provider == "Gemini (Google)":
    if custom_model:
        selected_model = custom_model
        base_url = f"https://generativelanguage.googleapis.com/v1beta/models/{custom_model}"
    else:
        selected_model = "gemini-2.0-flash"
        base_url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash"
else:  # ChatAI
    selected_model = custom_model if custom_model else "llama-3.3-70b-instruct"
    base_url = "https://chat-ai.academiccloud.de/v1"

# Konfiguration speichern
selected_config = {
    'provider': api_provider,
    'api_key': api_key,
    'base_url': base_url,
    'model': selected_model,
    'temperature': temperature,
    'granularity': granularity,
    'include_metadata': save_file_metadata,
    'verbose': show_debug_output
}

print(f"✓ Konfiguration gespeichert")
print(f"  Provider: {api_provider}")
print(f"  Modell: {selected_model}")
print(f"  Detailgrad: {granularity}")
print(f"  Temperatur: {temperature}")

In [ ]:
#@title **System-Prompt bearbeiten** (optional) { display-mode: "form" }
#@markdown Hier kannst du den System-Prompt anpassen.

#@markdown ### Eigenen System-Prompt verwenden:
use_custom_prompt = False #@param {type:"boolean"}

# Default-Prompt
DEFAULT_PROMPT = """# Prompt: Extraktion semantischer Triples aus frühneuzeitlichen Briefen  
**(LLM-optimiert, strikt themenzentriert, konzept-hierarchisch, nicht redundant)**

Du bist ein Experte für frühneuzeitliche Korrespondenz im **mitteleuropäischen Raum des späten 18. und frühen 19. Jahrhunderts (ca. 1750–1809)**.  
Deine Aufgabe ist die Extraktion **kanonischer, nicht redundanter** semantischer Triples  
(**Subjekt – Prädikat – Objekt**) aus historischen Briefen dieses Zeitraums.

Der **historische Hintergrund Mitteleuropas** (Bildungswesen, Konfessionen, soziale Ordnung, Elternautorität, Universitätskultur, Ehr- und Reputationsnormen, Briefkultur) ist **stillschweigend zu berücksichtigen**, ohne anachronistische Begriffe oder moderne Konzepte einzuführen.

Ziel ist ein **semantisch dichtes, stabiles, themenvergleichbares Ergebnis**, das sich für LLMs, Wissensgraphen und vergleichende Analysen eignet.

---

## INPUT
Du erhältst:
- `abstraktionslevel` (1–5)
- `brieftext` (vollständiger Text inkl. Briefkopf)
- TEI-XML, ignoriere den TEI-header
- ignoriere die Kommentare
- ignoriere die Editorial Notes

---

## OUTPUT (VERBINDLICH)
Gib **ausschließlich reines JSON** aus:

```json
{
  "entities": {...},
  "praedikate": {...},
  "triples": [...],
  "parameter": {...}
}
```

Keine Erklärungen, kein Markdown, keine Zusatzfelder.

---

## VERBINDLICHE NAMENSNORMALISIERUNG

Die folgenden historischen oder vollständigen Namensformen sind **immer** auf die kanonische Form zu normalisieren:

- Johann Paul Friedrich Richter → Jean Paul  
- Johann Wolfgang von Göthe → Goethe  
- Gotthold Ephraim Lessing → Lessing  
- Immanuel Kant → Kant  
- Friedrich Schiller → Schiller  

Weitere Varianten sind **analog zu vereinheitlichen**  
(bürgerlicher Vollname → etablierter Werkname).

---

## ABSTRAKTIONSLEVEL → ZIELANZAHL TRIPLES

| Level | Ziel |
|------|------|
| 1 | 1–2 Triples – Kernaussage |
| 2 | 3–5 Triples – Kernaussage + Hauptthemen |
| 3 | 8–12 Triples – Themen + Argumentstruktur |
| 4 | 12–20 Triples – thematisch relevante Details |
| 5 | 25+ Triples – implizite, klar ableitbare Bedeutungen |

---

## KERNPRINZIP: KONZEPT-HIERARCHIE STATT AKTEURS-GRAPH

Der Brief ist **primär als thematisches System** zu modellieren, nicht als Abfolge von Handlungen oder Personen.

### VERBINDLICHER MODELLIERUNGSPFAD

#### 1. OBERKONZEPTE IDENTIFIZIEREN  
Identifiziere zuerst **übergeordnete Themenfelder** (Oberkonzepte), z.B.:

- Emotion  
- Krankheit  
- Liebe  
- Freundschaft  
- Soziale Ordnung  
- Moral / Gewissen  
- Bildung  
- Lebensweg  
- Zukunft / Erwartung  

Diese werden als **Konzept-Entitäten (Typ: Konzept)** modelliert.

---

#### 2. SUBKONZEPTE ABLEITEN  
Leite daraus **inhaltlich unterscheidbare Unterthemen** ab.

Beispiele:
- Emotion → Schwermut, Angst, Hoffnung  
- Krankheit → Hypochondrie, körperliche Schwäche  
- Soziale Ordnung → elterliche Autorität, soziale Kontrolle, Ruf  

Subkonzepte sind **immer Konzepte, niemals Personenattribute**.

---

#### 3. KONZEPT–KONZEPT-BEZIEHUNGEN MODELLIEREN  
Modelliere bevorzugt Beziehungen **zwischen Ober- und Subkonzepten** oder zwischen Subkonzepten.

Beispiele:
- Schwermut ist_Subkonzept_von Emotion  
- Hypochondrie ist_Subkonzept_von Krankheit  
- Soziale_Kontrolle verschärft Emotionale_Krise  
- Krankheit beeinflusst Lebenswille  

**Mindestens 50 % aller Triples müssen Konzept↔Konzept sein.**

---

#### 4. PERSONEN NUR ALS TRÄGER (NACHRANGIG)
Personen dürfen **nur** erscheinen, wenn sie:
- Träger eines Konzepts sind
- von einem Konzept betroffen sind
- eine thematische Dynamik auslösen

Zulässige Muster:
- Krankheit betrifft Wagner  
- Emotionale_Krise betrifft Empfänger  

Unzulässig:
- Person ↔ Person ohne thematische Vermittlung

---

#### 5. ORTE NUR ALS THEMATISCHER RAHMEN (AUSNAHME)
Orte dürfen nur modelliert werden, wenn sie:
- einen thematischen Zustand rahmen oder beeinflussen
- sozial oder symbolisch relevant sind

Unzulässig:
- reine Lokalisierungen („schreibt aus X")

---

## AUFLÖSUNG VON ZUSTANDS-, ROLLEN- UND KOMPOSIT-ENTITÄTEN

Komplexe Zuschreibungen wie  
„X-Zustand von Person Y"  
dürfen **nicht** als eigene Entität stehen bleiben.

Sie sind **immer** zu zerlegen in:
1. ein **abstraktes Konzept**
2. eine **explizite Beziehung**

Beispiel:
- „Krankheitszustand Wagners"  
  → Konzept: Krankheit  
  → Triple: Krankheit betrifft Wagner

---

## REDUNDANZ-REGELN (STRIKT)

1. Keine Dubletten  
2. Keine Spiegelungen ohne Mehrwert  
3. Keine leeren Kommunikations-Triples  
4. Komplexe Sachverhalte als Ereignisknoten  
5. Verdichten statt auflisten  

---

## ENTITÄTEN

**Typen**:  
Person | Ort | Werk | Institution | Ereignis | Konzept | Zeitpunkt | Sonstiges  

**Normalisierung**:
- „ich" → Absender
- „Sie" → Empfänger
- moderne Rechtschreibung
- vollständige Datumsangaben
- historische Ortsnamen beibehalten

**IDs**: E1, E2, E3, …

---

## PRÄDIKATE

Verwende ein **kleines, stabiles Prädikatsinventar**  
(Level 3: ca. 10–16 Prädikate).

Beispiele:
- ist_Subkonzept_von
- beeinflusst
- verschärft
- beruhigt
- reguliert
- verursacht
- interpretiert_als
- betrifft
- unterliegt
- rahmt
- empfiehlt

---

## JSON-FORMAT (VERBINDLICH)

```json
{
  "entities": {
    "E1": {"label": "...", "typ": "..."}
  },
  "praedikate": {
    "P1": {"label": "...", "normalisiert_von": ["..."]}
  },
  "triples": [
    {"subjekt": "E1", "praedikat": "P1", "objekt": "E2"}
  ],
  "parameter": {
    "granularitaet": 3,
    "anzahl_triples": 10
  }
}
```

---

## START
Analysiere nun den bereitgestellten `brieftext` gemäß diesen Regeln  
und gib **ausschließlich das JSON** aus."""

if use_custom_prompt:
    print("📝 Bearbeite deinen System-Prompt:\n")
    
    import ipywidgets as widgets
    from IPython.display import display
    
    # Textarea Widget
    prompt_textarea = widgets.Textarea(
        value=DEFAULT_PROMPT,
        placeholder='System-Prompt hier eingeben...',
        description='',
        layout=widgets.Layout(width='100%', height='300px')
    )
    
    # Speichern-Button
    save_button = widgets.Button(
        description='✓ Prompt speichern',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    output_area = widgets.Output()
    
    def on_save_click(b):
        with output_area:
            output_area.clear_output()
            global CUSTOM_PROMPT
            CUSTOM_PROMPT = prompt_textarea.value
            print(f"✓ System-Prompt gespeichert ({len(CUSTOM_PROMPT)} Zeichen)")
    
    save_button.on_click(on_save_click)
    
    # Anzeigen
    display(prompt_textarea)
    display(save_button)
    display(output_area)
    
    # Initial speichern
    CUSTOM_PROMPT = DEFAULT_PROMPT
    
else:
    CUSTOM_PROMPT = None
    print("ℹ Standard-Prompt wird verwendet")

In [ ]:
#@title **Dateien hochladen** { display-mode: "form" }
#@markdown Lade deine XML-Dateien oder ein ZIP-Archiv hoch.

from google.colab import files
import zipfile
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output

# Upload-Verzeichnis
upload_dir = '/content/triple-colab/uploads'

# Globale Variable für Dateiliste
if 'uploaded_files' not in globals():
    uploaded_files = []

def reset_all():
    """Setzt alles zurück außer API-Konfiguration."""
    global uploaded_files, results, current_graph_index
    
    # Lösche Upload-Verzeichnis
    if os.path.exists(upload_dir):
        shutil.rmtree(upload_dir)
    os.makedirs(upload_dir, exist_ok=True)
    
    # Lösche Ergebnisse
    output_dir = '/content/triple-colab/output_json'
    if os.path.exists(output_dir):
        for file in os.listdir(output_dir):
            os.remove(os.path.join(output_dir, file))
    
    graphs_dir = '/content/triple-colab/graphs'
    if os.path.exists(graphs_dir):
        shutil.rmtree(graphs_dir)
    
    # Variablen zurücksetzen
    uploaded_files = []
    results = []
    current_graph_index = 0
    
    print("✓ Neustart durchgeführt - API-Konfiguration bleibt erhalten")

def upload_files():
    """Datei-Upload Handler."""
    global uploaded_files
    
    if uploaded_files:
        print("⚠ Es sind bereits Dateien hochgeladen. Nutze 'Neustart', um neue Dateien hochzuladen.")
        return
    
    # Upload-Verzeichnis sicherstellen
    os.makedirs(upload_dir, exist_ok=True)
    
    print("Bitte wähle deine Dateien aus...")
    uploaded = files.upload()
    
    # Hochgeladene Dateien verarbeiten
    for filename, content in uploaded.items():
        filepath = os.path.join(upload_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(content)
        
        # ZIP-Archive entpacken
        if filename.endswith('.zip'):
            print(f"📦 Entpacke {filename}...")
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(upload_dir)
            os.remove(filepath)
            print(f"✓ {filename} erfolgreich entpackt")
    
    # XML-Dateien sammeln
    for root, dirs, files_in_dir in os.walk(upload_dir):
        for file in files_in_dir:
            if file.endswith('.xml'):
                uploaded_files.append(os.path.join(root, file))
    
    update_file_list()

def delete_file(filepath):
    """Löscht eine einzelne Datei."""
    global uploaded_files
    
    if filepath in uploaded_files:
        uploaded_files.remove(filepath)
        if os.path.exists(filepath):
            os.remove(filepath)
    
    update_file_list()

def update_file_list():
    """Aktualisiert die Dateilisten-Anzeige."""
    with file_list_output:
        clear_output()
        
        if not uploaded_files:
            print("Keine Dateien hochgeladen.")
        else:
            print(f"✓ {len(uploaded_files)} XML-Datei(en):\n")
            
            for filepath in uploaded_files:
                filename = os.path.basename(filepath)
                
                # Lösch-Button für jede Datei
                delete_btn = widgets.Button(
                    description='🗑',
                    button_style='danger',
                    layout=widgets.Layout(width='50px', height='28px')
                )
                
                label = widgets.Label(value=filename)
                
                def make_delete_handler(fp):
                    def handler(b):
                        delete_file(fp)
                    return handler
                
                delete_btn.on_click(make_delete_handler(filepath))
                
                row = widgets.HBox([delete_btn, label])
                display(row)

# UI-Komponenten
upload_button = widgets.Button(
    description='📁 Dateien hochladen',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

reset_button = widgets.Button(
    description='🔄 Neustart',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

file_list_output = widgets.Output()

# Event Handler
upload_button.on_click(lambda b: upload_files())
reset_button.on_click(lambda b: (reset_all(), update_file_list()))

# UI anzeigen
display(widgets.HBox([upload_button, reset_button]))
display(file_list_output)

# Initial anzeigen
update_file_list()

In [ ]:
#@title **Verarbeitung starten** { display-mode: "form" }
#@markdown Startet die Triple-Extraktion für alle hochgeladenen Dateien.

import json
from datetime import datetime
from file_client import FileClient
from openwebui_client import OpenWebUIClient

# Validierung
if 'selected_config' not in globals():
    raise ValueError(
        "❌ Keine API-Konfiguration gefunden!\n\n"
        "Bitte führe zuerst die Zelle 'API-Konfiguration' aus."
    )

if 'api_key' not in globals() or not api_key:
    raise ValueError(
        "❌ Kein API-Schlüssel gefunden!\n\n"
        "Bitte führe die Zelle 'API-Konfiguration' aus und gib deinen API-Schlüssel ein."
    )

if 'uploaded_files' not in globals() or not uploaded_files:
    raise ValueError(
        "❌ Keine Dateien hochgeladen!\n\n"
        "Bitte führe zuerst die Zelle 'Dateien hochladen' aus."
    )

# Client initialisieren
client = OpenWebUIClient(
    api_key=selected_config['api_key'],
    base_url=selected_config['base_url'],
    model=selected_config['model'],
    temperature=selected_config['temperature']
)

# System-Prompt
system_prompt = CUSTOM_PROMPT if ('use_custom_prompt' in globals() and use_custom_prompt) else None

# Prompt-Template
def create_prompt(text, granularity):
    return f"""Extrahiere semantische Triples aus folgendem historischen Brieftext.

Granularität: {granularity}/10 (1=grob, 10=sehr detailliert)

Text:
{text}

Antworte NUR mit gültigem JSON im Format:
{{
  "triples": [
    {{"subject": "...", "predicate": "...", "object": "..."}}
  ]
}}"""

# Verarbeitung
results = []
output_dir = '/content/triple-colab/output_json'

print(f"Verarbeite {len(uploaded_files)} Datei(en)...\n")

for i, filepath in enumerate(uploaded_files, 1):
    filename = os.path.basename(filepath)
    print(f"[{i}/{len(uploaded_files)}] {filename}")
    
    try:
        # Text einlesen
        file_client = FileClient(filepath)
        text = file_client.read_content()
        
        if not text:
            print(f"  ⚠ Konnte Text nicht extrahieren")
            continue
        
        # API-Anfrage
        prompt = create_prompt(text, selected_config['granularity'])
        response = client.generate_response(prompt, system_prompt)
        
        if not response:
            print(f"  ❌ Keine Antwort erhalten")
            continue
        
        # JSON parsen
        try:
            clean_response = response.strip()
            if clean_response.startswith('```'):
                clean_response = '\n'.join(clean_response.split('\n')[1:-1])
            if clean_response.startswith('json'):
                clean_response = '\n'.join(clean_response.split('\n')[1:])
            
            data = json.loads(clean_response)
            triple_count = len(data.get('triples', []))
            
            # Ergebnis speichern
            output_filename = os.path.splitext(filename)[0] + '.json'
            output_path = os.path.join(output_dir, output_filename)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            results.append({
                'filename': filename,
                'triples': data.get('triples', []),
                'entities': data.get('entities', {}),
                'praedikate': data.get('praedikate', {}),
                'output_path': output_path
            })
            
            print(f"  ✓ {triple_count} Triples extrahiert")
            
        except json.JSONDecodeError as e:
            print(f"  ❌ JSON-Parse-Fehler: {e}")
            if selected_config.get('verbose', False):
                print(f"  Response: {response[:200]}...")
            continue
            
    except Exception as e:
        print(f"  ❌ Fehler: {e}")
        continue

print(f"\n✓ Verarbeitung abgeschlossen!")
print(f"  {len(results)} von {len(uploaded_files)} Dateien erfolgreich verarbeitet")
total_triples = sum(len(r['triples']) for r in results)
print(f"  {total_triples} Triples gesamt extrahiert")

In [ ]:
#@title **Ergebnis visualisieren** { display-mode: "form" }
#@markdown Zeigt die Wissensgraphen als durchblätterbare Galerie.

import plotly.graph_objects as go
import networkx as nx
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    # Graphs-Verzeichnis erstellen
    graphs_dir = '/content/triple-colab/graphs'
    os.makedirs(graphs_dir, exist_ok=True)
    
    # Aktueller Index (global für Navigation)
    if 'current_graph_index' not in globals():
        current_graph_index = 0
    
    def create_graph(result, save_png=True):
        """Erstellt einen Plotly-Graph aus Triples."""
        triples = result['triples']
        entities = result.get('entities', {})
        praedikate = result.get('praedikate', {})
        
        if not triples:
            return None, "⚠ Keine Triples zum Visualisieren gefunden."
        
        G = nx.DiGraph()
        
        # Hilfsfunktion: Entity-ID zu Label auflösen
        def get_label(entity_id):
            if entity_id in entities:
                return entities[entity_id].get('label', entity_id)
            return entity_id
        
        # Hilfsfunktion: Prädikat-ID zu Label auflösen
        def get_predicate_label(pred_id):
            if pred_id in praedikate:
                return praedikate[pred_id].get('label', pred_id)
            return pred_id
        
        for triple in triples:
            # Unterstütze beide Formate: deutsch (subjekt/objekt) und englisch (subject/object)
            subj_id = triple.get('subjekt', triple.get('subject', ''))
            pred_id = triple.get('praedikat', triple.get('predicate', ''))
            obj_id = triple.get('objekt', triple.get('object', ''))
            
            # IDs zu Labels auflösen
            subj = get_label(subj_id)
            pred = get_predicate_label(pred_id)
            obj = get_label(obj_id)
            
            if subj and obj:
                G.add_edge(subj, obj, label=pred)
        
        if len(G.nodes()) == 0:
            return None, "⚠ Keine Knoten im Graph gefunden."
        
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Edge-Traces
        edge_trace = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_trace.append(
                go.Scatter(
                    x=[x0, x1, None],
                    y=[y0, y1, None],
                    mode='lines',
                    line=dict(width=1, color='#888'),
                    hoverinfo='none',
                    showlegend=False
                )
            )
        
        # Node-Trace
        node_x = []
        node_y = []
        node_text = []
        
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_text.append(node)
        
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            text=node_text,
            textposition='top center',
            hoverinfo='text',
            marker=dict(
                size=20,
                color='lightblue',
                line=dict(width=2, color='darkblue')
            ),
            showlegend=False
        )
        
        # Figure erstellen
        fig = go.Figure(
            data=edge_trace + [node_trace],
            layout=go.Layout(
                title=f"Wissensgraph: {result['filename']}",
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=20, r=20, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                height=600,
                width=800
            )
        )
        
        # PNG speichern
        if save_png:
            try:
                png_filename = os.path.splitext(result['filename'])[0] + '.png'
                png_path = os.path.join(graphs_dir, png_filename)
                fig.write_image(png_path, width=1200, height=800)
                result['graph_path'] = png_path
            except Exception as e:
                # PNG-Export fehlgeschlagen, aber Graph kann trotzdem angezeigt werden
                pass
        
        stats = f"Graph mit {len(G.nodes())} Entitäten und {len(G.edges())} Beziehungen"
        return fig, stats
    
    def show_graph(index):
        """Zeigt Graph an gegebenem Index."""
        with graph_output:
            clear_output(wait=True)
            
            if 0 <= index < len(results):
                result = results[index]
                fig, stats = create_graph(result)
                
                if fig:
                    fig.show()
                    print(f"\n{stats}")
                else:
                    print(stats)
            
            # Navigation-Status aktualisieren
            nav_label.value = f"Datei {index + 1} von {len(results)}"
            prev_btn.disabled = (index == 0)
            next_btn.disabled = (index == len(results) - 1)
    
    def on_prev_click(b):
        global current_graph_index
        if current_graph_index > 0:
            current_graph_index -= 1
            show_graph(current_graph_index)
    
    def on_next_click(b):
        global current_graph_index
        if current_graph_index < len(results) - 1:
            current_graph_index += 1
            show_graph(current_graph_index)
    
    # Navigation-Buttons
    prev_btn = widgets.Button(
        description='← Zurück',
        button_style='info',
        layout=widgets.Layout(width='120px')
    )
    
    next_btn = widgets.Button(
        description='Weiter →',
        button_style='info',
        layout=widgets.Layout(width='120px')
    )
    
    nav_label = widgets.Label(
        value=f"Datei 1 von {len(results)}",
        layout=widgets.Layout(width='150px')
    )
    
    prev_btn.on_click(on_prev_click)
    next_btn.on_click(on_next_click)
    
    graph_output = widgets.Output()
    
    # UI anzeigen
    nav_box = widgets.HBox([prev_btn, nav_label, next_btn], layout=widgets.Layout(justify_content='center'))
    display(nav_box)
    display(graph_output)
    
    # Ersten Graph anzeigen
    show_graph(current_graph_index)

In [ ]:
#@title **Ergebnisse als ZIP herunterladen** { display-mode: "form" }
#@markdown Lädt alle JSON-Ergebnisse und PNG-Grafiken als ZIP-Archiv herunter.

from google.colab import files
import zipfile
from datetime import datetime

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f'triple_extraction_results_{timestamp}.zip'
    zip_path = f'/content/{zip_filename}'
    
    graphs_dir = '/content/triple-colab/graphs'
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # JSON-Dateien hinzufügen
        for result in results:
            output_path = result['output_path']
            arcname = f"json/{os.path.basename(output_path)}"
            zipf.write(output_path, arcname)
        
        # PNG-Grafiken hinzufügen (falls vorhanden)
        if os.path.exists(graphs_dir):
            for result in results:
                if 'graph_path' in result and os.path.exists(result['graph_path']):
                    graph_path = result['graph_path']
                    arcname = f"graphs/{os.path.basename(graph_path)}"
                    zipf.write(graph_path, arcname)
    
    print(f"Lade {zip_filename} herunter...")
    print(f"  • {len(results)} JSON-Dateien")
    
    graph_count = sum(1 for r in results if 'graph_path' in r and os.path.exists(r['graph_path']))
    if graph_count > 0:
        print(f"  • {graph_count} PNG-Grafiken")
    
    files.download(zip_path)
    print(f"\n✓ Download gestartet")

## Hilfe & Troubleshooting

| Problem | Lösung |
|---------|--------|
| **NameError: api_key not defined** | Führe die Zelle "API-Konfiguration" aus |
| **Ungültiger API-Schlüssel** | Prüfe ob der Schlüssel korrekt kopiert wurde |
| **Rate Limit Error** | Warte kurz und versuche es erneut |
| **Timeout** | Versuche es mit weniger oder kleineren Dateien |
| **Keine Triples extrahiert** | Erhöhe den Detailgrad oder passe den Prompt an |
| **JSON Parse Error** | Aktiviere "Debug-Modus" in der Konfiguration |

**API-Schlüssel erhalten:**
- Gemini: https://aistudio.google.com/apikey
- ChatAI: https://chat-ai.academiccloud.de/
- OpenAI: https://platform.openai.com/api-keys